# Debug

During a call with Casey and Jim, they noticed 2 unusual things in the generic_gene_summary table:
* Not all values in the `num_simulated` column were equal to 25, which should be the case
* There are some genes that do not have any DE statistics reported. This is the case using a template experiment that is included in the recount2 training compendium, so its not just an issue using an external template experiment.

These two issues were NOT observed in the other generic_gene_summary tables training compendium = Powers et. al. or Pseudomonas datasets.

See [example summary tables](https://docs.google.com/spreadsheets/d/1aqSPTLd5bXjYOBoAjG7bM8jpy5ynnCpG0oBC2zGzozc/edit?usp=sharing)

Given that this is only observed in the recount2 training dataset, we suspect that this is an issue with mapping genes from ensembl ids (raw data) to hgnc ids (needed to compare against validation dataset)

In [35]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

import os
import re
import pandas as pd
from ponyo import utils
from generic_expression_patterns_modules import process, new_experiment_process

In [2]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

# Read in config variables
config_filename = os.path.abspath(
    os.path.join(base_dir, "configs", "config_human_general.tsv")
)

params = utils.read_config(config_filename)

In [3]:
# Load params
# Output files of recount2 template experiment data
raw_template_recount2_filename = params['raw_template_filename']
mapped_template_recount2_filename = os.path.join(
    base_dir, 
    "human_general_analysis",
    params['processed_template_filename']
)

# Local directory to store intermediate files
local_dir = params['local_dir']

# ID for template experiment
# This ID will be used to label new simulated experiments
project_id = params['project_id']

In [4]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

# Read in config variables
config_filename = os.path.abspath(
    os.path.join(base_dir, "configs", "config_new_experiment.tsv")
)

params = utils.read_config(config_filename)

In [5]:
# Load params
# Output files of recount2 template experiment data
raw_template_nonrecount2_filename = params['raw_template_filename']
mapped_template_nonrecount2_filename = params['processed_template_filename']

## Read data

In [6]:
# Read raw template data
# This is the data **before** gene ids were mapped
raw_template_recount2 = pd.read_csv(raw_template_recount2_filename, sep="\t", index_col=0, header=0)
raw_template_nonrecount2 = pd.read_csv(raw_template_nonrecount2_filename, sep="\t", index_col=0, header=0).T

# Read mapped template data
# This is the data **after** gene ids were mapped
mapped_template_recount2 = pd.read_csv(mapped_template_recount2_filename, sep="\t", index_col=0, header=0)
mapped_template_nonrecount2 = pd.read_csv(mapped_template_nonrecount2_filename, sep="\t", index_col=0, header=0)

In [7]:
print(raw_template_recount2.shape)
raw_template_recount2.head()

(36, 58037)


,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,ENSG00000001036.13,ENSG00000001084.10,ENSG00000001167.14,...,ENSG00000283690.1,ENSG00000283691.1,ENSG00000283692.1,ENSG00000283693.1,ENSG00000283694.1,ENSG00000283695.1,ENSG00000283696.1,ENSG00000283697.1,ENSG00000283698.1,ENSG00000283699.1
SRR493937,622,1,398,394,154,10661,5865,1217,764,1061,...,0,3,0,0,0,0,22,25,0,0
SRR493938,622,0,399,362,159,10761,5770,1184,744,981,...,0,2,0,0,0,0,31,19,0,0
SRR493939,3077,0,629,911,503,941,12913,1768,694,1553,...,0,2,0,0,0,1,17,18,0,0
SRR493940,3041,0,694,918,476,886,12732,1773,695,1540,...,0,2,0,0,0,0,14,18,0,0
SRR493941,551,1,411,563,226,2303,5917,1232,625,963,...,0,1,0,0,0,0,22,33,0,0


In [8]:
print(mapped_template_recount2.shape)
mapped_template_recount2.head()

(24, 17755)


,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A4GALT,A4GNT,AAAS,AACS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
SRR493937,244,396,6,144077,2089,7,946,7,941,752,...,214,161,202,569,1618,9,768,12758,5060,773
SRR493938,230,384,8,142807,2062,8,968,3,978,776,...,219,171,215,576,1655,7,808,12925,5061,783
SRR493939,308,396,11,77651,1064,4,321,2,1633,1518,...,345,372,198,387,1315,30,842,4339,3304,743
SRR493940,303,351,13,77739,1125,12,325,0,1637,1481,...,318,317,155,448,1322,36,795,4400,3308,714
SRR493941,203,327,0,235645,2438,6,945,15,1520,1027,...,191,71,240,533,2006,4,1331,14318,4776,936


In [9]:
print(raw_template_nonrecount2.shape)
raw_template_nonrecount2.head()

(72, 58528)


,TSPAN6,TNMD,DPM1,SCYL3,C1ORF112,FGR,CFH,FUCA2,GCLC,NFYA,...,RP11-165N12.2,RP11-964E11.3,BMS1P21,LLNLR-245B6.1,CTC-325H20.8,RP11-1437A8.7,CTD-2060L22.1,RP11-107E5.4,RARRES2P11,RP11-299P2.2
sample10A.genes.results,915.98,0.0,2619.0,921.81,769.91,8.0,3580.03,3939.24,3547.12,2113.0,...,1.00,6.0,0.0,263.60,0.0,0.0,0.0,0.0,0.0,0.0
sample10B.genes.results,925.00,0.0,2299.0,969.24,526.39,7.0,2454.45,3497.36,3313.71,1291.0,...,0.00,5.0,0.0,238.80,0.0,0.0,0.0,0.0,0.0,0.0
sample10C.genes.results,567.00,0.0,1646.0,434.32,687.28,1.0,1047.58,3116.40,2144.00,1355.0,...,0.00,6.0,0.0,109.59,0.0,0.0,1.0,0.0,0.0,0.0
sample11A.genes.results,1314.03,0.0,2620.0,736.22,1119.47,3.0,3112.37,3845.86,2564.49,1434.0,...,1.00,19.0,0.0,142.34,0.0,0.0,2.0,0.0,0.0,0.0
sample11B.genes.results,1225.00,0.0,2196.0,839.88,931.16,15.0,6182.22,2576.73,2992.04,1331.0,...,1.83,49.0,0.0,187.18,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
print(mapped_template_nonrecount2.shape)
mapped_template_nonrecount2.head()

(6, 17755)


,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A4GALT,A4GNT,AAAS,AACS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
sample10A.genes.results,0.001527,0.008043,0.000000,0.000000e+00,0.00003,0.001502,0.009762,0.0,0.018250,0.057363,...,0.014476,0.038087,0.001794,0.002161,0.031684,0.005665,0.017585,0.153839,0.015464,0.024786
sample10B.genes.results,0.001607,0.007614,0.000030,0.000000e+00,0.00003,0.001153,0.008697,0.0,0.014671,0.052432,...,0.011555,0.029453,0.001681,0.001944,0.030478,0.005184,0.014875,0.132834,0.014160,0.018387
sample10C.genes.results,0.000647,0.004115,0.000015,0.000000e+00,0.00001,0.000498,0.003677,0.0,0.011238,0.030207,...,0.012232,0.042378,0.000572,0.001221,0.018776,0.002039,0.007589,0.101177,0.008307,0.013707
sample9A.genes.results,0.000909,0.004631,0.000000,0.000000e+00,0.00001,0.000119,0.004032,0.0,0.020424,0.028384,...,0.016189,0.032129,0.000972,0.001512,0.017863,0.004940,0.010900,0.110883,0.012866,0.015768
sample9B.genes.results,0.002114,0.005322,0.000015,8.337300e-07,0.00003,0.000126,0.003119,0.0,0.019631,0.028471,...,0.017950,0.035351,0.001457,0.001805,0.016249,0.004162,0.009575,0.115659,0.009989,0.019448


## Look up some genes

## Case 1:
Genes have only simulated statistics using recount2 template experiment:
* ENSG00000169717 --> ACTRT2
* ENSG00000184895 --> SRY

In [11]:
ensembl_ids = ["ENSG00000169717",
               "ENSG00000184895",
               "ENSG00000124232",
               "ENSG00000261713",
               "ENSG00000186818",
               "ENSG00000160882"               
              ]

ensembl_version_ids = raw_template_recount2.columns

for sub in ensembl_ids:
    for x in ensembl_version_ids:
        if re.search(sub, x):
            print(x)

ENSG00000169717.6
ENSG00000184895.7
ENSG00000124232.10
ENSG00000261713.6
ENSG00000186818.12
ENSG00000160882.11


In [12]:
raw_template_recount2[["ENSG00000169717.6", "ENSG00000184895.7"]].sum()

ENSG00000169717.6    2
ENSG00000184895.7    0
dtype: int64

In [13]:
mapped_template_recount2[["ACTRT2","SRY"]].sum()

ACTRT2    0
SRY       0
dtype: int64

Looks like reason for missing values in template experiment could be due to having all 0 counts

## Case 2:

Genes that have all statistics present but number of simulated experiments < 25 using recount2 template. These genes are also only have simulated statistics using non-recount2 template experiment:
* ENSG00000186818 --> LILRB4
* ENSG00000160882 --> CYP11B1

In [14]:
raw_template_nonrecount2[["LILRB4","CYP11B1"]].sum()

LILRB4     26.53
CYP11B1     2.00
dtype: float64

In [15]:
mapped_template_nonrecount2[["LILRB4","CYP11B1"]].sum()

LILRB4     0.0
CYP11B1    0.0
dtype: float64

So far, it seems that those genes that are missing template statistics have all 0 counts in the template experiment.

In [16]:
raw_template_recount2[["ENSG00000186818.12",
                       "ENSG00000160882.11"]].sum()

ENSG00000186818.12    32182
ENSG00000160882.11       15
dtype: int64

In [17]:
mapped_template_recount2[["LILRB4","CYP11B1"]].sum()

LILRB4     32126
CYP11B1        9
dtype: int64

Overall there isn't a trend found in these genes missing some number of simulated experiments, so let's try looking at the simulated experiments. At this point we suspect that the missing simulated experiments are those where genes have all 0 counts.

In [18]:
# Get list of files
simulated_dir = os.path.join(
    local_dir,
    "pseudo_experiment",
)

simulated_filename_list = []
for file in os.listdir(simulated_dir):
    if (project_id in file) and ("simulated" in file) and ("encoded" not in file):
        simulated_filename_list.append(os.path.join(simulated_dir,file))

In [19]:
assert len(simulated_filename_list) ==25

In [20]:
# For each simulated experiment, check how many have all 0 counts for this gene
# Is this number the same number of missing simulated experiments?
counter_LILRB4 = 0
counter_CYP11B1 = 0
for filename in simulated_filename_list:
    simulated_data = pd.read_csv(filename, sep="\t", index_col=0, header=0)
    if simulated_data["LILRB4"].sum() == 0:
        counter_LILRB4 += 1
    if simulated_data["CYP11B1"].sum() == 0:
        counter_CYP11B1 += 1
        
# Verified LILRB4 to be missing 2 experiments (23 total experiments)
# Verified CYP11B1 to be missing 8 experiments (17 total experiments)
# Can look this up in google sheet
print(counter_LILRB4, counter_CYP11B1)

2 8


## Case 3:

Genes that do not have a p-value using non-recount2 template experiment:
* ENSG00000124232 --> RBPJL
* ENSG00000261713 --> SSTR5-AS1

Following the theme observed in Case 1 and 2, we suspect that these "missing" p-values actually indicate p-value =0

In [21]:
# Look up these values in DE statis output files
DE_stats_dir = os.path.join(
    local_dir,
    "DE_stats"
)

template_nonrecount2_DE_stats_filename = os.path.join(
    DE_stats_dir,
    "DE_stats_template_data_cis-par-KU1919_real.txt"
)

template_nonrecount2_DE_stats = pd.read_csv(
    template_nonrecount2_DE_stats_filename,
    sep="\t",
    index_col=0,
    header=0
)

template_nonrecount2_DE_stats.loc["RBPJL"]

baseMean          1.199346
log2FoldChange    3.645592
lfcSE             2.675809
stat              1.362426
pvalue            0.173063
padj                   NaN
Name: RBPJL, dtype: float64

In [22]:
template_nonrecount2_DE_stats.loc["SSTR5.AS1"]

baseMean          0.600980
log2FoldChange    2.584014
lfcSE             3.488476
stat              0.740729
pvalue            0.458858
padj                   NaN
Name: SSTR5.AS1, dtype: float64

According to this [link](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#pvaluesNA), genes with NA adjusted p-values indicate those genes that have been automatically filtered by DESeq2 as very likely to not have no significance. 

Let's test calling DESeq2, turning off filtering

In [23]:
transposed_template_filename = "/home/alexandra/Documents/Data/Generic_expression_patterns/Costello_BladderCancer_ResistantCells_Counts_12-8-20_transposed.txt"

new_experiment_process.transpose_save(raw_template_nonrecount2_filename, transposed_template_filename)

In [29]:
project_id = "cis-par-KU1919"
local_dir = "/home/alexandra/Documents/Data/Generic_expression_patterns/"
mapped_compendium_filename = "/home/alexandra/Documents/Data/Generic_expression_patterns/mapped_recount2_compendium.tsv"

In [27]:
# Check that the feature space matches between template experiment and VAE model.  
# (i.e. ensure genes in template and VAE model are the same).
mapped_template_experiment = new_experiment_process.compare_match_features(
    transposed_template_filename,
    mapped_compendium_filename
)
mapped_template_filename = transposed_template_filename

(72, 58528)
(49651, 17755)


In [30]:
# Load metadata file with processing information
sample_id_metadata_filename = os.path.join(
    "data",
    "metadata",
    f"{project_id}_process_samples.tsv"
)

# Read in metadata
metadata = pd.read_csv(sample_id_metadata_filename, sep='\t', header=0, index_col=0)

# Get samples to be dropped
sample_ids_to_drop = list(metadata[metadata["processing"] == "drop"].index)

In [31]:
# Modify template experiment
process.subset_samples_template(
    mapped_template_filename,
    sample_ids_to_drop,
)

In [32]:
process.recast_int_template(mapped_template_filename)

In [33]:
# Load metadata file with grouping assignments for samples
metadata_filename = os.path.join(
    "data",
    "metadata",
    f"{project_id}_groups.tsv"
)

# Check whether ordering of sample ids is consistent between gene expression data and metadata
process.compare_and_reorder_samples(mapped_template_filename, metadata_filename)

sample ids are ordered correctly


In [39]:
%%R -i metadata_filename -i project_id -i mapped_template_filename -i local_dir -i base_dir
library("limma")
library("DESeq2")
# Manually change DESeq2 call with additional parameter

get_DE_stats_DESeq <- function(metadata_file,
                               experiment_id,
                               expression_file,
                               data_type,
                               local_dir,
                               run) {

  # This function performs DE analysis using DESeq.
  # Expression data in expression_file are grouped based on metadata_file
  #
  # Arguments
  # ---------
  # metadata_file: str
  #   File containing mapping between sample id and group
  #
  # experiment_id: str
  #   Experiment id used to label saved output filee
  #
  # expression_file: str
  #   File containing gene expression data
  #
  # data_type: str
  #   Either 'template' or 'simulated' to label saved output file
  #
  # local_dir: str
  #   Directory to save output files to
  #
  # run: str
  #   Used as identifier for different simulated experiments

  expression_data <- t(as.matrix(read.csv(expression_file, sep="\t", header=TRUE, row.names=1)))
  metadata <- as.matrix(read.csv(metadata_file, sep="\t", header=TRUE, row.names=1))

  print("Checking sample ordering...")
  print(all.equal(colnames(expression_data), rownames(metadata)))

  group <- interaction(metadata[,1])

  mm <- model.matrix(~0 + group)

  #print(head(expression_data))

  ddset <- DESeqDataSetFromMatrix(expression_data, colData=metadata, design = ~group)

  deseq_object <- DESeq(ddset)

  deseq_results <- results(deseq_object, independentFiltering=FALSE)

  deseq_results_df <-  as.data.frame(deseq_results)

  # Save summary statistics of DEGs
  if (data_type == "template") {
    out_file = paste(local_dir, "DE_stats/DE_stats_template_data_", experiment_id,"_", run, ".txt", sep="")
  } else if (data_type == "simulated") {
    out_file = paste(local_dir, "DE_stats/DE_stats_simulated_data_", experiment_id,"_", run, ".txt", sep="")
  }
  write.table(deseq_results_df, file = out_file, row.names = T, sep = "\t", quote = F)
}


# File created: "<local_dir>/DE_stats/DE_stats_template_data_SRP012656_real.txt"
get_DE_stats_DESeq(metadata_filename,
                   project_id, 
                   mapped_template_filename,
                   "template",
                   local_dir,
                   "real_without_filtering")

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: estimating size factors

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: estimating dispersions

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: gene-wise dispersion estimates

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: mean-dispersion relationship

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: final dispersion estimates

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_ex

[1] "Checking sample ordering..."
[1] TRUE


In [40]:
template_nonrecount2_nofilter_DE_stats_filename = os.path.join(
    DE_stats_dir,
    "DE_stats_template_data_cis-par-KU1919_real_without_filtering.txt"
)

template_nonrecount2_nofilter_DE_stats = pd.read_csv(
    template_nonrecount2_nofilter_DE_stats_filename,
    sep="\t",
    index_col=0,
    header=0
)

template_nonrecount2_nofilter_DE_stats.loc["RBPJL"]

baseMean          1.199346
log2FoldChange    3.645592
lfcSE             2.675809
stat              1.362426
pvalue            0.173063
padj              0.380897
Name: RBPJL, dtype: float64

In [41]:
template_nonrecount2_nofilter_DE_stats.loc["SSTR5.AS1"]

baseMean          0.600980
log2FoldChange    2.584014
lfcSE             3.488476
stat              0.740729
pvalue            0.458858
padj              0.708907
Name: SSTR5.AS1, dtype: float64

**Takeaways:**
* Case 1: genes with only simulated statistics is because template experiment has all 0 counts
* Case 2: genes with fewer than 25 simulated experiments, some simulated experiments have all 0 counts for those genes
* Case 3 (only found using non-recount2 template): genes with missing p-value in template experiment have NaN output from DESeq2, which indicates those genes that have been automatically filtered by DESeq2 as very likely to not have no significance: https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#pvaluesNA


**Proposed solution:**
* Case 1 and 2: Remove genes with 0 counts across all samples. I will also add an option to additional remove genes that have mean counts < user specified threshold (for Jim's case). This should get rid of those rows with missing values in template statistic columns. These genes will be missing in the validation against Crow et. al. so I'll need to ignore these genes in the rank comparison. Any gene removed from the simulated experiments will get a lower `number of simulated experiments` reported so I will need to document this for the user.
* Case 3: (option 1) I can change the parameter setting to turn off this autofiltering and it will perform all tests and report them. (option 2) Replace padj values = "NA" with "Filtered by DESeq2" and document that this means that DESeq2 pre-filtered these genes as likely not being signicant to help increase detection power. For now I am using option 1. DESeq2 documentation states: 

*filter out those tests from the procedure that have no, or little chance of showing significant evidence, without even looking at their test statistic. Typically, this results in increased detection power at the same experiment-wide type I error, as measured in terms of the false discovery rate.* 

*For weakly expressed genes, we have no chance of seeing differential expression, because the low read counts suffer from so high Poisson noise that any biological effect is drowned in the uncertainties from the read counting*